In [1]:
%matplotlib notebook

import os
import shutil

import pyBigWig
import pybedtools

import pandas as pd
import numpy as numpy

import re

from os import listdir
from os.path import isfile, join

# Tornado plot with featureCounts

In [2]:
experiment_id = "NS-23.0061"

metadata_subdir = os.path.expanduser("~/fht.samba.data/experiments/ATACseq/{}/analysis/metadata/".format(experiment_id))
print('metadata_subdir exists:{}\n'.format(os.path.exists(metadata_subdir)))

bw_dir = os.path.expanduser("~/fht.samba.data/experiments/ATACseq/{}/alignment/coverage/".format(experiment_id))
print('bw_dir exists:{}\n'.format(os.path.exists(bw_dir)))

bed_dir = '../beds/'
print('bed_dir exists:{}\n'.format(os.path.exists(bed_dir)))

metadata_subdir exists:True

bw_dir exists:True

bed_dir exists:True



In [3]:
def create_dir(name_dir):
    if os.path.exists(name_dir):
        shutil.rmtree(name_dir)
    os.mkdir(name_dir)
    return name_dir

In [4]:
TP_dir = create_dir('../TP/')
TP_dir

'../TP/'

In [5]:
def read_contrast(metadata_subdir, experiment_id): 
    
    metadata_files = [f for f in listdir(metadata_subdir) if isfile(join(metadata_subdir, f))]
    print('Files in metadata subdirectory:\n{}\n'.format(metadata_files))

    input_orig_contrast_dir = list(filter(lambda x: ('_contrasts_r' in x and experiment_id in x), metadata_files))[0] 
    contrasts_df = pd.read_table(metadata_subdir + input_orig_contrast_dir, delimiter = '\t', index_col = 0)
    print('input_orig_contrast_dir: {} - shape: {} \n'.format(input_orig_contrast_dir, contrasts_df.shape))  
    
    return contrasts_df

contrasts_df = read_contrast(metadata_subdir, experiment_id)
contrasts_df

Files in metadata subdirectory:
['ATACseq_ARID1AKO_ARID1BKD.csv', 'ATACseq_ARID1AKO_ARID1BKD.txt', 'NS-23.0061_metadata_r12x34.txt', 'ATACseq_ARID1AKO_ARID1BKD.xlsx', 'NS-23.0061_group_dict.json', 'NS-23.0061_contrast_dict.json', 'NS-23.0061_contrasts_r5x3.txt']

input_orig_contrast_dir: NS-23.0061_contrasts_r5x3.txt - shape: (5, 3) 



,test_group,negative_ctrl_grp,file_friendly_name
contrast,,,
ARID1BKD_HCT116 - WT_HCT116,ARID1BKD_HCT116,WT_HCT116,ARID1BKD_HCT116_WT_HCT116
ARID1AKO_HCT116 - WT_HCT116,ARID1AKO_HCT116,WT_HCT116,ARID1AKO_HCT116_WT_HCT116
ARID1AKO_ARID1BKD_HCT116 - WT_HCT116,ARID1AKO_ARID1BKD_HCT116,WT_HCT116,ARID1AKO_ARID1BKD_HCT116_WT_HCT116
ARID1AKO_ARID1BKD_HCT116 - ARID1AKO_HCT116,ARID1AKO_ARID1BKD_HCT116,ARID1AKO_HCT116,ARID1AKO_ARID1BKD_HCT116_ARID1AKO_HCT116
ARID1BKD_TOV21G - WT_TOV21G,ARID1BKD_TOV21G,WT_TOV21G,ARID1BKD_TOV21G_WT_TOV21G


In [ ]:
def computeMatrix(contrasts_df, bed_dir, bw_dir, TP_dir):
    
  
    for contrast in contrasts_df.index[:]:

        friendly_name = contrasts_df[contrasts_df.index==contrast]['file_friendly_name'][0]
        print("\ncontrast:  {}, friendly_name:  {}".format(contrast, friendly_name))
        
        negative_ctrl_grp = contrasts_df[contrasts_df.index==contrast]['negative_ctrl_grp'][0]
        test_group = contrasts_df[contrasts_df.index==contrast]['test_group'][0]
        print("\nnegative_ctrl_grp:  {}, \ntest_group:  {} ".format(negative_ctrl_grp, test_group))        
        
        # For DMSO
        for direction in ['down', 'unchanged', 'up']:
            print("\ndirection:  {}".format(direction))
            !computeMatrix reference-point -R {bed_dir}ATAC_seq_{friendly_name}_{direction}.bed  -S {bw_dir}{negative_ctrl_grp}.bw  --samplesLabel {friendly_name}  --outFileName {TP_dir}ATAC_seq_{friendly_name}_{direction}_{negative_ctrl_grp}.dat  --outFileSortedRegions {TP_dir}ATAC_seq_{friendly_name}_{direction}_{negative_ctrl_grp}.bed --referencePoint center   --beforeRegionStartLength 1000 --afterRegionStartLength 1000  --sortRegions 'descend' --sortUsing 'mean'  --averageTypeBins 'mean' --binSize 20  --transcriptID transcript  --exonID exon --transcript_id_designator transcript_id 
         
        !cat {TP_dir}ATAC_seq_{friendly_name}_down_{negative_ctrl_grp}.bed {TP_dir}ATAC_seq_{friendly_name}_unchanged_{negative_ctrl_grp}.bed {TP_dir}ATAC_seq_{friendly_name}_up_{negative_ctrl_grp}.bed > {TP_dir}ATAC_seq_{friendly_name}_all_{negative_ctrl_grp}.bed            
        
        cmd_computeMatrix_DMSO = """computeMatrix reference-point  -R {TP_dir}ATAC_seq_{friendly_name}_all_{negative_ctrl_grp}.bed   -S {bw_dir}{negative_ctrl_grp}.bw  {bw_dir}{test_group}.bw  --outFileName {TP_dir}ATAC_seq_{friendly_name}.dat  --referencePoint center   --beforeRegionStartLength 1000 --afterRegionStartLength 1000  --sortRegions 'keep' --sortUsing 'mean'  --averageTypeBins 'mean' --binSize 20  --transcriptID transcript  --exonID exon --transcript_id_designator transcript_id""".format(TP_dir=TP_dir, friendly_name=friendly_name, bw_dir=bw_dir, negative_ctrl_grp=negative_ctrl_grp, test_group=test_group)
        # print(cmd_computeMatrix_DMSO)
        !{cmd_computeMatrix_DMSO}
        
computeMatrix(contrasts_df, bed_dir, bw_dir, TP_dir)


contrast:  ARID1BKD_HCT116 - WT_HCT116, friendly_name:  ARID1BKD_HCT116_WT_HCT116

negative_ctrl_grp:  WT_HCT116, 
test_group:  ARID1BKD_HCT116 

direction:  down

direction:  unchanged

direction:  up

contrast:  ARID1AKO_HCT116 - WT_HCT116, friendly_name:  ARID1AKO_HCT116_WT_HCT116

negative_ctrl_grp:  WT_HCT116, 
test_group:  ARID1AKO_HCT116 

direction:  down

direction:  unchanged

direction:  up

contrast:  ARID1AKO_ARID1BKD_HCT116 - WT_HCT116, friendly_name:  ARID1AKO_ARID1BKD_HCT116_WT_HCT116

negative_ctrl_grp:  WT_HCT116, 
test_group:  ARID1AKO_ARID1BKD_HCT116 

direction:  down

direction:  unchanged

direction:  up


In [ ]:
def plotHeatmap(contrasts_df, bed_dir, bw_dir, TP_dir):   

    for contrast in contrasts_df.index[:]:

        friendly_name = contrasts_df[contrasts_df.index==contrast]['file_friendly_name'][0]
        print("\ncontrast:  {}, friendly_name:  {}".format(contrast, friendly_name))
        
        negative_ctrl_grp = contrasts_df[contrasts_df.index==contrast]['negative_ctrl_grp'][0]
        test_group = contrasts_df[contrasts_df.index==contrast]['test_group'][0]
        print("\nnegative_ctrl_grp:  {}, \ntest_group:  {} ".format(negative_ctrl_grp, test_group))        
                
        cmd_plotHeatmap = """plotHeatmap --matrixFile {TP_dir}ATAC_seq_{friendly_name}.dat --outFileName {TP_dir}ATAC_seq_{friendly_name}.pdf  --colorMap Oranges   --plotFileFormat 'pdf'    --dpi '500'  --sortRegions 'no'  --sortUsing 'mean'  --averageTypeSummaryPlot 'mean'  --plotType 'lines'  --missingDataColor 'black'   --alpha '1.0'    --xAxisLabel '' --yAxisLabel ''  --heatmapWidth 7.5 --heatmapHeight 25.0  --whatToShow 'plot, heatmap and colorbar'  --startLabel '' --endLabel ''  --refPointLabel '0'       --legendLocation 'best'  --labelRotation '0'""".format(TP_dir=TP_dir, friendly_name=friendly_name, contrast=contrast)
        # print(cmd_plotHeatmap)
        ! {cmd_plotHeatmap}
    
plotHeatmap(contrasts_df, bed_dir, bw_dir, TP_dir)